In [1]:
import os
import librosa
import Loader
import numpy as np
import librosa.display
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
def Trim(Signal,Sr,Len = 5):
    Signal = np.array(Signal[:(Len*Sr)])
    return Signal
    

def Pad(Signal,Len,Sr = 22050):
    if len(Signal) < (Len*Sr):
        Pad_Size = (Len*Sr)-len(Signal)
        Padded = np.pad(Signal, (0,Pad_Size ), 'constant')
        return Padded
    return Signal

def Waveform(Signal,Sr):
    librosa.display.waveplot(Signal,sr=Sr)
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.show()


def FFT(Signal,Sr):
    fft = np.fft.fft(Signal)
    magnitude = np.abs(fft)
    freq = np.linspace(0,Sr,len(magnitude))
    freq = freq[:int(len(freq)/2)]
    magnitude = magnitude[:int(len(magnitude)/2)]
    plt.plot(freq,magnitude)
    plt.xlabel("Frequency")
    plt.ylabel("Magnitude")
    plt.show()



def Sepctogram(Signal,Sr,Mode = 'Normal'): #Using STFT
    Sample_Num = 2048 #Per fft
    Hop_length = 512 
    stft = librosa.core.stft(Signal,hop_length=Hop_length,n_fft=Sample_Num)
    spect = np.abs(stft)
    if Mode == 'log':
        spect = librosa.amplitude_to_db(spect)
    librosa.display.specshow(spect,sr=Sr,hop_length = Hop_length)
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.colorbar()
    plt.show()

def MFCC(Signal,Sr):
    Sample_Num = 2048 #Per fft
    Hop_length = 512 
    MFCC = librosa.feature.mfcc(Signal,sr=Sr,n_fft=Sample_Num,hop_length = Hop_length,n_mfcc=20)
    librosa.display.specshow(MFCC,sr=Sr,hop_length = Hop_length)
    plt.xlabel("Time")
    plt.ylabel("MFCC")
    plt.colorbar()
    plt.show()

def Save_MFCC(Signal,Sr,n_mfcc=13,n_fft=2048,hop_length=512):
    MFCC = librosa.feature.mfcc(Signal,sr=Sr,n_fft=n_fft,hop_length = hop_length,n_mfcc=n_mfcc)
    return MFCC/303.0

In [4]:
Pain_Path,Hungry_Path,Uncomfortable_Path = Loader.LOAD()
Audio_Hungry = []
Audio_Pain = []
Audio_UnCom = []
for path in Hungry_Path:
    Signal,Sr = librosa.load(path,sr=22050) #len(Signal) = sr * AudioTime  
    Audio_Hungry.append(Trim(Pad(Signal,5,Sr),Sr))

for path in Pain_Path:
    Signal,Sr = librosa.load(path,sr=22050) #len(Signal) = sr * AudioTime  
    Audio_Pain.append(Trim(Pad(Signal,5,Sr),Sr))

for path in Uncomfortable_Path:
    Signal,Sr = librosa.load(path,sr=22050) #len(Signal) = sr * AudioTime  
    Audio_UnCom.append(Trim(Pad(Signal,5,Sr),Sr))

print('Audio Loaded')

Audio Loaded


In [5]:
np.random.shuffle(Audio_Pain)
np.random.shuffle(Audio_UnCom)
np.random.shuffle(Audio_Hungry)

In [6]:
Data = []
Y = []

for i in range(len(Audio_UnCom) + 100):
    Data.append(Save_MFCC(Audio_Hungry[i],Sr))
    Y.append(0)

for Audio_File in Audio_Pain:
    Data.append(Save_MFCC(Audio_File,Sr))
    Y.append(1)

for Audio_File in Audio_UnCom:
    Data.append(Save_MFCC(Audio_File,Sr))
    Y.append(2)

Data = np.array(Data)
Y = np.array(Y)


In [8]:
Split = int(len(Data)*0.90)
X_Train = Data[:Split]
Y_Train = Y[:Split]
X_Test = Data[Split:]
Y_Test = Y[Split:]

In [9]:
Model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(20,5,activation='relu',padding='same',input_shape = (13,216)),
        tf.keras.layers.MaxPool1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation= 'relu'),
    tf.keras.layers.Dense(256,activation= tf.keras.activations.elu),
    tf.keras.layers.Dense(128,activation='relu'),   
    tf.keras.layers.Dense(3,activation='softmax')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
Model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics = ["accuracy"])
Hist = Model.fit(X_Train,Y_Train,epochs=25,batch_size=6,validation_data=(X_Test,Y_Test))


Epoch 1/25
83/83 [==============================] - 1s 13ms/step - loss: 0.9509 - accuracy: 0.5932 - val_loss: 1.5412 - val_accuracy: 0.2909
Epoch 2/25
83/83 [==============================] - 0s 2ms/step - loss: 0.7769 - accuracy: 0.7410 - val_loss: 1.3064 - val_accuracy: 0.3818
Epoch 3/25
83/83 [==============================] - 0s 2ms/step - loss: 0.7274 - accuracy: 0.7101 - val_loss: 0.7326 - val_accuracy: 0.5273
Epoch 4/25
83/83 [==============================] - 0s 2ms/step - loss: 0.7106 - accuracy: 0.7410 - val_loss: 2.0311 - val_accuracy: 0.0909
Epoch 5/25
83/83 [==============================] - 0s 2ms/step - loss: 0.6522 - accuracy: 0.7521 - val_loss: 1.1924 - val_accuracy: 0.4545
Epoch 6/25
83/83 [==============================] - 0s 2ms/step - loss: 0.5286 - accuracy: 0.7962 - val_loss: 1.4097 - val_accuracy: 0.5636
Epoch 7/25
83/83 [==============================] - 0s 2ms/step - loss: 0.5514 - accuracy: 0.8086 - val_loss: 0.9892 - val_accuracy: 0.6182
Epoch 8/25
83/83 [=

In [10]:
Model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 13, 20)            21620     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 6, 20)             0         
_________________________________________________________________
flatten (Flatten)            (None, 120)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               61952     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 3

In [18]:
Name = input('Name: ')
Model.save(Name + '.h5')
